In [ ]:
import pandas as pd
import numpy as np
import datetime
import sys, os
import glob
import time
import fnmatch
import os
import datetime
import xlwings as xw
from datetime import date
start_time = time.time()

a=fnmatch.filter(os.listdir('.'), 'ZCM UPLOAD*')
a=a[0]

zcmuploads = pd.read_excel(a,dtype={'*Zebra PN': str, '*Zebra Site': str,'*Supplier ID': str}, sheet_name='WEEKLY- Net Reqs')

zcmuploads['*Date'] = pd.to_datetime(zcmuploads['*Date']).dt.date
zcmuploads.head()

#get date of june 30 2022
july1 = datetime.date(2022, 6, 30)

#identify infor before july
mask1 = (zcmuploads['*Date'] <= july1)
zcmuploads['month'] = np.where(mask1, 'beforejuly','afterjuly')
zcmuploads.head()

#zcmuploads = zcmuploads[(zcmuploads['*Transaction Type'] == 'ONHANDWK') | (zcmuploads['*Transaction Type'] == 'Net Requirements') | 
                       #(zcmuploads['*Transaction Type'] == 'GROSSFRCTWK') | 
                       #(zcmuploads['*Transaction Type'] == 'ONHANDDLY')] #delete extra rows

zcmuploads.drop(labels=['Flex String PIT 1','Flex String PIT 2','Flex String PIT 3'], axis=1, inplace = True)
zcmuploads.rename(columns={'*Quantity': 'Quantity'},inplace=True)
zcm_onhand_dly= zcmuploads[(zcmuploads['*Transaction Type']=='GROSSFRCTWK')]

In [300]:
zcmuploads.head()

,#*Zebra,*Supplier ID,*Zebra PN,Supplier PN,*Zebra Site,*Transaction Type,Quantity,*Date,month
0,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX2,BOOKEDSODLY,4,2022-05-16,beforejuly
1,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX2,ONHANDDLY,279,2022-05-16,beforejuly
2,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX2,ONHANDWK,279,2022-05-16,beforejuly
3,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX2,GROSSFRCTWK,2,2022-05-20,beforejuly
4,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX2,RMADLY,182,2022-06-03,beforejuly


In [288]:
#sum of forecast
zcmuploadspiv=zcm_onhand_dly.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
print(len(zcmuploadspiv))
#zcmuploadspiv['*Transaction Type'] ='ONHANDWK'
zcmuploadspiv.head()

3


Quantity           
month                          afterjuly beforejuly
*Zebra PN         *Supplier ID                     
TC83BH-2205A710RW 24063            313.0       10.0
TC84              24063             30.0        4.0
                  26389              0.0        5.0

# FINAL TABLE OH TX2 BEFORE JULY

In [289]:
#one part number per quarter and supplier
zcm_onhand_dly12= zcm_onhand_dly.drop(['*Transaction Type'], axis=1)
zcm_onhand_dly12=zcm_onhand_dly.drop_duplicates(subset=('*Zebra PN','month','*Supplier ID'),keep='first')
zcm_onhand_dly12.head()

#assign corresponding OH TO rows that will be uploaded
zcm_onhand_dly12 = pd.merge(zcm_onhand_dly12,zcmuploadspiv, on =['*Zebra PN','*Supplier ID'])
zcm_onhand_dly12.head()

#zcm_onhand_dly12=zcm_onhand_dly12[zcm_onhand_dly12['*Transaction Type']== 'GROSSFRCTWK']
zcm_onhand_dly12['Quantity'] = np.where(zcm_onhand_dly12['month'] == 'beforejuly',zcm_onhand_dly12[('Quantity','beforejuly')],zcm_onhand_dly12[('Quantity', 'afterjuly')]) 
zcm_onhand_dly12= zcm_onhand_dly12.drop([('Quantity','beforejuly'),('Quantity','afterjuly')], axis = 1)
zcm_onhand_dly12

zcm_onhand_dly13=zcm_onhand_dly12[zcm_onhand_dly12['month']== 'beforejuly']
zcm_onhand_dly13['*Transaction Type']= 'ONHANDWKTX2'
zcm_onhand_dly13['*Zebra Site'] = 'TX2'
zcm_onhand_dly13['*Date']=date.today()
zcm_onhand_dly13

C:\Users\MJ7255\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
<ipython-input-289-353883ca0d00>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zcm_onhand_dly13['*Transaction Type']= 'ONHANDWKTX2'
<ipython-input-289-353883ca0d00>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zcm_onhand_dly13['*Zebra Site'] = 'TX2'
<ipython-input-289-353883ca

,#*Zebra,*Supplier ID,*Zebra PN,Supplier PN,*Zebra Site,*Transaction Type,Quantity,*Date,month
0,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX2,ONHANDWKTX2,10.0,2022-05-25,beforejuly
2,Zebra,26389,TC84,8014-019229,TX2,ONHANDWKTX2,5.0,2022-05-25,beforejuly
3,Zebra,24063,TC84,8014-019229,TX2,ONHANDWKTX2,4.0,2022-05-25,beforejuly


In [290]:
strings = time.strftime("%m.%d.%Y")
stringstx2= 'OHTX2 ' + strings 
b=("%s.xlsx" % stringstx2)

with pd.ExcelWriter(b) as writer:  
       zcm_onhand_dly13.to_excel(writer, index= False)

* TX2 THERE WILL BE NO UPLOADS FOR NET REQ

# OH  TX4 (AFTER JULY)

In [291]:
zcm_onhand_now= zcmuploads[(zcmuploads['*Transaction Type']=='ONHANDWK')] #edit
zcm_onhand_now
#sum of oh after july
zcm_oh_now_piv=zcm_onhand_now.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
print(len(zcm_oh_now_piv))
zcm_oh_now_piv.head()

2


Quantity           
month                          afterjuly beforejuly
*Zebra PN         *Supplier ID                     
TC83BH-2205A710RW 24063             45.0      280.0
TC84              24063              0.0        9.0

In [292]:
#one part number per quarter, delete column tt so if it coccurs on either oh prev july or frcst the row will be consider
zcm_net_req = zcmuploads.drop(['*Transaction Type'], axis=1)
zcm_net_req =zcm_net_req.drop_duplicates(subset=('*Zebra PN','month','*Supplier ID'),keep='first')
print(zcm_net_req.shape,'shape')
zcm_net_req.head()

#get the total sum of current OH before and after july per row
zcm_net_req0=pd.merge(zcm_net_req,zcm_oh_now_piv, on =['*Zebra PN','*Supplier ID'],how='left')
zcm_net_req0[('Quantity', 'afterjuly')] = zcm_net_req0[('Quantity', 'afterjuly')].fillna(0)
zcm_net_req0[('Quantity', 'beforejuly')] = zcm_net_req0[('Quantity', 'beforejuly')].fillna(0)
print(len(zcm_net_req0))
zcm_net_req0

#DELETE duplicated columns here
zcm_net_req0.drop(labels=['Quantity','*Zebra Site','#*Zebra','Supplier PN'], axis=1, inplace = True)
zcm_onhand_dly13.drop(labels=['*Date','month'],axis=1,inplace= True)

#merge quantity used for july (forecast before july, oh to upload for TX2)
zcm_net_req1 = pd.merge(zcm_net_req0,zcm_onhand_dly13, on =['*Zebra PN','*Supplier ID'],how='left')

zcm_net_req1['OHTX4'] = zcm_net_req1[('Quantity', 'afterjuly')] + zcm_net_req1[('Quantity', 'beforejuly')] - zcm_net_req1['Quantity']
zcm_net_req1.drop(labels=[('Quantity', 'afterjuly'),('Quantity', 'beforejuly'),'Quantity'], axis = 1, inplace = True)
zcm_net_req1= zcm_net_req1[zcm_net_req1['month']=='afterjuly']
zcm_net_req1

(5, 8) shape
5


C:\Users\MJ7255\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\MJ7255\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,*Supplier ID,*Zebra PN,*Date,month,#*Zebra,Supplier PN,*Zebra Site,*Transaction Type,OHTX4
1,24063,TC83BH-2205A710RW,2022-07-01,afterjuly,Zebra,8014-019230,TX2,ONHANDWKTX2,315.0
4,24063,TC84,2023-09-11,afterjuly,Zebra,8014-019229,TX2,ONHANDWKTX2,5.0


In [278]:
strings = time.strftime("%m.%d.%Y")
stringstx4= 'OHTX4 ' + strings 
b=("%s.xlsx" % stringstx4)

with pd.ExcelWriter(b) as writer:  
       zcm_net_req1.to_excel(writer, index= False)

# NET REQ TX4

In [293]:
forecast_afterjuly=zcm_onhand_dly12[zcm_onhand_dly12['month']== 'afterjuly']
#to avoid duplicate columns, keep net rq to upload
forecast_afterjuly.drop(labels=['*Date','*Transaction Type','*Zebra Site','#*Zebra','Supplier PN','month'],axis=1,inplace = True)
forecast_afterjuly

C:\Users\MJ7255\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,*Supplier ID,*Zebra PN,Quantity
1,24063,TC83BH-2205A710RW,313.0
4,24063,TC84,30.0


In [294]:
#sum just measures to extract from forecast, #edit
rest_to_forecast= zcmuploads[(zcmuploads['*Transaction Type']=='INTERNALREQDLY') | (zcmuploads['*Transaction Type'] == 'RMADLY')]
rest_to_fcst_piv=rest_to_forecast.pivot_table(index=('*Zebra PN','*Supplier ID'), columns='month', aggfunc='sum').fillna(0)
print(len(rest_to_fcst_piv))
#zcmuploadspiv['*Transaction Type'] ='ONHANDWK'
rest_to_fcst_piv.head()

2


Quantity           
month                          afterjuly beforejuly
*Zebra PN         *Supplier ID                     
TC83BH-2205A710RW 24063             10.0      182.0
TC84              24063              5.0        0.0

In [295]:
#Net req to upload
#what NET REQ we will upload? those PN in after july, before july wias already considered 
net_req_tx4_0= zcmuploads[(zcmuploads['month']=='afterjuly')]
#just one part umber per month 
net_req_tx4_0.drop(labels=['*Date','*Transaction Type','month','Quantity'],axis=1, inplace = True)
net_req_tx4_0 = net_req_tx4_0.drop_duplicates()
net_req_tx4_0
#zcm_onhand_dly13.drop(labels=['*Date','month'],axis=1,inplace= True)

#add forecast per row
net_req_tx4= pd.merge(net_req_tx4_0, forecast_afterjuly, on =['*Zebra PN','*Supplier ID'],how='left')
net_req_tx4.rename(columns={'Quantity': 'Forecast'},inplace=True)
net_req_tx4

#add qty to substract(sum RMA, Intransit)
net_req_tx4= pd.merge(net_req_tx4, rest_to_fcst_piv, on =['*Zebra PN','*Supplier ID'],how='left')
net_req_tx4.drop(labels=[('Quantity', 'beforejuly')],axis=1,inplace= True)
net_req_tx4.rename(columns={('Quantity', 'afterjuly'):'comming'}, inplace= True)

#add OHTX4 afterjuly
oh_tx4 = zcm_net_req1
oh_tx4.drop(labels=['*Date','month','#*Zebra','Supplier PN','*Zebra Site','*Transaction Type'],axis=1, inplace= True)

net_req_tx4= pd.merge(net_req_tx4,oh_tx4, on =['*Zebra PN','*Supplier ID'],how='left')
net_req_tx4['comming'].fillna(0, inplace=True)
net_req_tx4['oh+wip+all'] = net_req_tx4['comming']+net_req_tx4['OHTX4']
net_req_tx4['Net req'] = net_req_tx4['Forecast'] - net_req_tx4['oh+wip+all']
net_req_tx4

net_req_tx4['Net req'] = np.where(net_req_tx4['Net req'] < 0, 0, net_req_tx4['Net req'])
net_req_tx4.drop(labels=['comming', 'OHTX4'],axis=1,inplace= True)
net_req_tx4=net_req_tx4.drop_duplicates() 
net_req_tx4

C:\Users\MJ7255\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\MJ7255\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,#*Zebra,*Supplier ID,*Zebra PN,Supplier PN,*Zebra Site,Forecast,oh+wip+all,Net req
0,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX4,313.0,325.0,0.0
1,Zebra,24063,TC84,8014-019229,TX4,30.0,10.0,20.0


In [296]:
net_req_tx4_date=zcm_onhand_dly[zcm_onhand_dly['month']=='afterjuly']
net_req_tx4_date['Cummulative_fcst'] = net_req_tx4_date.groupby(['*Zebra PN','*Supplier ID'])['Quantity'].cumsum(axis = 0)
net_req_tx41=net_req_tx4.drop(labels = ['#*Zebra','*Zebra Site','Supplier PN'], axis=1)
net_req_tx4_date

<ipython-input-296-c786074be246>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  net_req_tx4_date['Cummulative_fcst'] = net_req_tx4_date.groupby(['*Zebra PN','*Supplier ID'])['Quantity'].cumsum(axis = 0)


,#*Zebra,*Supplier ID,*Zebra PN,Supplier PN,*Zebra Site,*Transaction Type,Quantity,*Date,month,Cummulative_fcst
11,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX4,GROSSFRCTWK,3,2022-07-01,afterjuly,3
13,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX4,GROSSFRCTWK,7,2022-07-15,afterjuly,10
15,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX4,GROSSFRCTWK,7,2022-07-22,afterjuly,17
16,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX4,GROSSFRCTWK,6,2022-07-29,afterjuly,23
17,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX4,GROSSFRCTWK,6,2022-08-05,afterjuly,29
...,...,...,...,...,...,...,...,...,...,...
90,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX4,GROSSFRCTWK,7,2023-09-15,afterjuly,313
101,Zebra,24063,TC84,8014-019229,TX4,GROSSFRCTWK,8,2023-09-11,afterjuly,8
102,Zebra,24063,TC84,8014-019229,TX4,GROSSFRCTWK,2,2023-10-09,afterjuly,10
103,Zebra,24063,TC84,8014-019229,TX4,GROSSFRCTWK,12,2023-10-16,afterjuly,22


In [297]:
net_req_tx4_date1 = pd.merge(net_req_tx4_date,net_req_tx41, on =['*Zebra PN','*Supplier ID'],how='left')
net_req_tx4_date1.tail()

,#*Zebra,*Supplier ID,*Zebra PN,Supplier PN,*Zebra Site,*Transaction Type,Quantity,*Date,month,Cummulative_fcst,Forecast,oh+wip+all,Net req
61,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX4,GROSSFRCTWK,7,2023-09-15,afterjuly,313,313.0,325.0,0.0
62,Zebra,24063,TC84,8014-019229,TX4,GROSSFRCTWK,8,2023-09-11,afterjuly,8,30.0,10.0,20.0
63,Zebra,24063,TC84,8014-019229,TX4,GROSSFRCTWK,2,2023-10-09,afterjuly,10,30.0,10.0,20.0
64,Zebra,24063,TC84,8014-019229,TX4,GROSSFRCTWK,12,2023-10-16,afterjuly,22,30.0,10.0,20.0
65,Zebra,24063,TC84,8014-019229,TX4,GROSSFRCTWK,8,2023-10-23,afterjuly,30,30.0,10.0,20.0


In [298]:
net_req_tx4_date1['Net_req1']=np.where(net_req_tx4_date1['Net req']>0,np.where(net_req_tx4_date1['Cummulative_fcst']<net_req_tx4_date1['Net req'],
                                                                               0,net_req_tx4_date1['Quantity']),0)
net_req_tx4_date1.drop(labels=['month','Forecast','oh+wip+all','Net req'],axis=1,inplace = True)
net_req_tx4_date1.tail()

,#*Zebra,*Supplier ID,*Zebra PN,Supplier PN,*Zebra Site,*Transaction Type,Quantity,*Date,Cummulative_fcst,Net_req1
61,Zebra,24063,TC83BH-2205A710RW,8014-019230,TX4,GROSSFRCTWK,7,2023-09-15,313,0
62,Zebra,24063,TC84,8014-019229,TX4,GROSSFRCTWK,8,2023-09-11,8,0
63,Zebra,24063,TC84,8014-019229,TX4,GROSSFRCTWK,2,2023-10-09,10,0
64,Zebra,24063,TC84,8014-019229,TX4,GROSSFRCTWK,12,2023-10-16,22,12
65,Zebra,24063,TC84,8014-019229,TX4,GROSSFRCTWK,8,2023-10-23,30,8


In [299]:
stringsnr= 'Net_Requirements ' + strings 
b=("%s.xlsx" % stringsnr)

with pd.ExcelWriter(b) as writer:  
       net_req_tx4_date1.to_excel(writer, index= False)

In [132]:
#TX4 OH will be available from 8 jul, too add to oh (tx2 after july)